<a href="https://colab.research.google.com/github/Yukoock/Diplomado-2017/blob/master/4_4_Review_Products_Gem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reviews App

In [1]:
%%capture
!pip install langchain-openai
!pip install gradio
!pip install langchain-google-genai

In [3]:
import os
from google.colab import userdata
api_key = userdata.get('GEMINI')

In [4]:
import gradio as gr
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
import os

# Plantilla para extracción de información
template = """\
Analiza el siguiente texto y extrae la siguiente información:

sentimiento: ¿Cuál es el tono emocional del contenido?
Responde Positivo, Negativo, Neutral o Desconocido.

tiempo_entrega: ¿Se menciona el tiempo de entrega o disponibilidad?
Indica el número de días/horas si se especifica, sino "Sin información".

valoración_precio: ¿Cómo percibe el autor el precio/costo?
Responde Caro, Económico, Justo o Desconocido.

recomendación: ¿Recomendaría el producto/experiencia?
Responde Sí, No, Neutral o Desconocido.

tema_principal: ¿Cuál es el tema principal del texto? (Máximo 3 palabras)

Formatea la salida como un resumen estructurado en español con estos puntos:
- Análisis de sentimiento
- Tiempo de entrega/espera
- Percepción del valor
- Recomendación
- Tema destacado

texto: {review}
"""

prompt = PromptTemplate(
    input_variables=["review"],
    template=template)

def extract_key_data(review: str) -> str:
    try:
        # 1. Validar entrada vacía
        if not review or not review.strip():
            return "⚠️ Por favor, ingresa un texto para analizar."

        # 2. Validar longitud de la entrada
        if len(review.split()) > 700:
            return "⚠️ Por favor, ingresa un texto más corto. La longitud máxima es de 700 palabras."

        # 3. Verificar API key antes de inicializar LLM
        if not api_key:
            return "❌ Error: No se encontró GOOGLE_API_KEY. Por favor configura tu API key de Google Generative AI."

        # 4. Inicializar el LLM
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            google_api_key=api_key)

        # 5. Crear el parser de salida
        parser = StrOutputParser()

        # 6. Crear la cadena (Chain)
        chain = prompt | llm | parser

        # 7. Invocar la cadena
        result = chain.invoke({"review": review})

        # 8. Devolver el resultado
        return result

    except Exception as e:
        return f"❌ Error al procesar el texto: {str(e)}\n\nVerifica que tu API key de Google esté configurada correctamente."


In [5]:
# Interfaz Gradio
iface = gr.Interface(
    fn = extract_key_data,
    inputs=[
        gr.Textbox(
            label="Texto a analizar",
            lines=10,
            placeholder="Pega aquí reseñas, comentarios en redes, artículos...",
            info="Puede ser sobre productos, experiencias, servicios o contenido digital")],
    outputs=gr.Textbox(
        label="Análisis completo",
        lines=15,
        interactive=False),
    title="🛍️📱 Analizador de Opiniones 2.0",
    description="""Extrae insights clave de cualquier texto: reseñas, comentarios en redes, feedback de clientes...
    Detecta sentimientos, percepción de valor y temas principales automáticamente.""",
    examples=[
        ["El nuevo smartphone es increíble aunque algo caro. Llegó en 2 días. Lo recomiendo totalmente!"],
        ["La serie empezó bien pero el final fue decepcionante. No vale la pena pagar la suscripción."],
        ["El restaurante tiene ambiente agradable pero los precios son elevados para la calidad de la comida."]],
    theme="soft",
    allow_flagging="never")

iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://de97703f436d843fc5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Ejemplos:

"¡Me encantó este producto! Llegó súper rápido, en solo 2 días después de hacer el pedido. La calidad es excelente y funciona perfectamente para lo que necesitaba. Además, el precio me pareció muy razonable comparado con otras opciones. ¡Totalmente recomendado!"

"Qué decepción. El envío tardó una eternidad, más de 15 días en llegar cuando prometieron 5. Al abrirlo, el producto tenía un rasguño visible y no funciona como esperaba. No volvería a comprar."

"El producto en sí está bien, cumple su función básica. Sin embargo, me parece demasiado caro para la calidad que ofrece, esperaba algo más robusto por ese coste. No recuerdo cuánto tardó en llegar, pero el embalaje era adecuado."

"Estoy muy contento con la compra. Llegó bastante rápido. El producto es tal cual se describe y el precio está en línea con lo que esperaba pagar por algo así. Funciona bien hasta ahora."

"Recibí el artículo y es funcional. Hace lo que dice que hace, ni más ni menos. No tengo quejas particulares sobre el proceso."